# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

сalls — количество звонков,

minutes — суммарная длительность звонков в минутах,

messages — количество sms-сообщений,

mb_used — израсходованный интернет-трафик в Мб,

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Откройте и изучите файл

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Данные загружены. Они не содержат пропусков, типы данных приведены в соответствие с самими данными. С такими данными можно работать без предварительной обработки.

## Разбейте данные на выборки

Целевой признак, который нам необходимо предсказать - это выбор тарифа пользователем. В датасете обозначается переменной 'is_ultra'. Остальные переменные будут признаками для обучения модели, это количество звонков (calls), минут разговоров (minutes), сообщений (messages) и использованных мб интернета (mb_used).

In [4]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Разобьем нашу выборку данных на обучающую, валидационную и тестовую в следующем процентном соотношении: 60% - 20% - 20%.

In [5]:
train_ratio = 0.60
validation_ratio = 0.20
test_ratio = 0.20

# сначала отделяем тренировочную выборку
features_train, features_test, target_train, target_test =\
train_test_split(features, target, test_size=1 - train_ratio,
                 random_state=12345)

#делим оставшиеся данные на тестовую и валидационную выборки
features_valid, features_test, target_valid, target_test =\
train_test_split(features_test, target_test, 
                 test_size=test_ratio/(test_ratio + validation_ratio),
                 random_state=12345)


In [6]:
print('Размер обучающей выборки:', features_train.shape[0])
print('Размер валидационной выборки:', features_valid.shape[0])
print('Размер тестовой выборки:', features_test.shape[0])

Размер обучающей выборки: 1928
Размер валидационной выборки: 643
Размер тестовой выборки: 643


## Исследуйте модели

Целевой признак наших данных может принимать всего два значения - 0 и 1, где 0 - это тариф "Смарт", а 1 - тариф "Ультра". Таким образом перед нами стоит задача бинарной классификации. Для выбора наилучшей модели протестируем следующие виды моделей подходящие для подобного рода задач:

- Решающее дерево (Decision Tree);
- Случайный лес (Random Forest);
- Логистическая регрессия (Logistic Regression)

Модели мы будем обучать на обучающей выборки, а проверять ее работу на валидационной выборке с помощью метрики Accuracy, равной отношению числа правильных ответов к размеру тестовой выборки.

### Модель решающего дерева

Основным гиперпараметром определяющим работу данной модели, является значение глубины (max_depth). Найдем такое значение этого гиперпараметра, при котором модель покажет наибольшее значение метрики Accuracy:

In [7]:
best_result = 0
for depth in range(1,30):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result_valid = model.score(features_valid, target_valid)
    if result_valid > best_result:
        best_result = result_valid
        best_model_tree = model
        best_max_depth = depth
        
print('Лучший результат показала модель с глубиной', best_max_depth, 'и accuracy', best_result)

Лучший результат показала модель с глубиной 3 и accuracy 0.7853810264385692


Как видим, в данной задаче нам потребовалась совсем небольшая глубина модели для достижения наилучшего результата. При большей глубине модель уже начинает переобучаться и выдавать достойный результат только на обучающей выборке.

### Модель случайного леса

Основными гиперпараметрами определяющими работу данной модели, являются значение глубины (max_depth) и количества деревьев (n_estimators). Найдем такие значения этих гиперпараметров, при которых модель покажет наибольшее значение метрики Accuracy:

In [8]:
range_est = range(10, 51, 5)
range_depth = range (1, 10) 
best_result = 0

for est in range_est:
    for depth in range_depth:
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        result_valid = model.score(features_valid, target_valid)
        if result_valid > best_result:
            best_result = result_valid
            best_model_forest = model
            best_max_depth = depth
            best_est = est
        
print('Лучший результат показала модель с глубиной', best_max_depth, 'и количеством деревьев',
      best_est, ', acuuracy равно', best_result)

Лучший результат показала модель с глубиной 8 и количеством деревьев 40 , acuuracy равно 0.8087091757387247


Ожидаемо, случайны лес показал лучший результат нежели у модели решающего дерева. Однако гиперпараметры при этом выбраны не самые оптимальные для скорости работы такой модели. Если бы наш датасет был не такой маленький, то максимальные величины гиперпараметров модели случайного леса, при которых модель обучается за адекватное время, были бы для нас куда меньше.

### Модель логистической регрессии

Эта модель не склонна к переобучению, основными гиперпараметрами являются максимальное количество итераций (max_iter) и выбор решателя (solver). поскольку у нас небольшой набор данных и все модели обучаются быстро, поставим максимальное значение итераций достаточно большим, что бы все варианты моделей сошлись. Выбор параметра solvers предлагает нам следующие варианты:

- liblinear,
- newton-cg (этот параметр на наших данных выдает ошибку не сходимости решения, поэтому его уберем из эксперимента),
- lbfgs,
- sag,
- saga.

Для небольших наборов данных рекомендуется использовать параметр 'lbfgs', для больших 'sag' или 'saga'. Но мы выясним на практике какой из них лучше подойдет для нашей задачи. Найдем такое значение гиперпараметра solver, при котором модель покажет наибольшее значение метрики Accuracy:

In [9]:
best_result = 0
for solver in ['liblinear', 'lbfgs', 'sag', 'saga']:
    model = LogisticRegression(random_state=12345, solver=solver, max_iter=5000)
    model.fit(features_train, target_train)
    result_valid = model.score(features_valid, target_valid)
    if result_valid > best_result:
        best_result = result_valid
        best_model_logistic = model
        best_solver = solver

print("Лучший результат показала модель с solver =", best_solver, 'accuracy равно', best_result)

Лучший результат показала модель с solver = lbfgs accuracy равно 0.7107309486780715


Таким образом лучший результат в модели логистической регрессии показал решатель 'lbfgs', который согласно документации лучше подходит для малых объемов данных.

**Вывод по исследованию моделей:**

Согласно метрике качества accuracy. наилучший результат из всех моделей показала модель случайного леса с гиперпараметрами n_estimators=40, max_depth=8.

Протестируем эту модель на тестовой выборке.

## Проверьте модель на тестовой выборке

In [10]:
result_test = best_model_forest.score(features_test, target_test)
print('Результат лучшей модели случайного леса на тестовой выборке, accuracy =', result_test)

Результат лучшей модели случайного леса на тестовой выборке, accuracy = 0.7962674961119751


Результат на тестовой выборке получился слегка меньше чем на валидационной, однако все равно больше чем у остальных тестируемых моделей и больше необходимого нам по условиям задачи минимального значения 0.75.

## (бонус) Проверьте модели на адекватность

Проверим какой результат показала бы случайная модель дающая в качестве предсказания всегда одно и то же значение - 0 и проверим ее точность на тестовой выборке:

In [11]:
predictions = pd.Series(0, index=target_test.index)
accuracy_score(target_test, predictions)

0.6842923794712286

Точность такой модели говорит нам о том что распределение значений целевого признака в выборке смещено в сторону значения 0. И если мы всем новым пользователям будем давать тариф "Смарт", мы угадаем с выбором в 68% случаев. Значение accuracy нашей наилучшей модели почти 80%, что говорит о ее адекватности. Однако если бы мы остановили свой выбор на модели логистической регрессии, то вероятность была бы очень близка к вероятности нашей случайной модели, а значит смысла в такой модели на практике не много. 

# Вывод по проекту

Для задачи подбора подходящего тарифа было протестировано три типа модели классификации: решающее дерево, случайный лес и логистическая регрессия. Опытным путем определено, что наилучший результат дает модель случайного леса с гиперпараметрами n_estimators=40, max_depth=8. На тестовой выборке эта модель показала значение Accuracy = 0.796. Это значение больше минимального допустимого по условиям задачи 0.75, а значит модель подходит для решения поставленной задачи. 